### Used for Train Test Split before augmentation

In [72]:
import json
import os
from PIL import Image, ImageDraw

In [70]:
with open("../data/coco.json") as f: #. in scripts, .. in notebook
    original_annotations = json.load(f) 

with open("../data/coco_sliced_coco.json") as f:
    sliced_annotations = json.load(f)

# Initialise an empty dictionary for adjusted annotations
adjusted_annotations = {"images": [], "annotations": []}

print(original_annotations)
print(sliced_annotations)
print(adjusted_annotations)


{'images': [{'width': 4096, 'height': 2000, 'id': 0, 'file_name': '01BE01.bmp'}], 'annotations': [{'id': 0, 'image_id': 0, 'category_id': 6, 'segmentation': [], 'bbox': [3243.732519135956, 674.1576397397976, 96.69150425145112, 116.35757291276306], 'ignore': 0, 'iscrowd': 0, 'area': 11250.788755982963}, {'id': 1, 'image_id': 0, 'category_id': 6, 'segmentation': [], 'bbox': [2752.15197481034, 1706.6478091354281, 45.88749354306201, 80.30311370035747], 'ignore': 0, 'iscrowd': 0, 'area': 3684.9086114129277}, {'id': 2, 'image_id': 0, 'category_id': 6, 'segmentation': [], 'bbox': [1529.1878991321566, 557.4169134727985, 45.887493543061424, 37.693298267514876], 'ignore': 0, 'iscrowd': 0, 'area': 1729.6509808672772}, {'id': 3, 'image_id': 0, 'category_id': 6, 'segmentation': [], 'bbox': [1567.6792100432908, 505.00420582112133, 23.630943165701463, 29.886192827210394], 'ignore': 0, 'iscrowd': 0, 'area': 706.2389241390035}, {'id': 4, 'image_id': 0, 'category_id': 6, 'segmentation': [], 'bbox': [170

In [87]:
# Helper function to calculate intersection and adjust bounding box
def check_and_adjust_bbox(bbox, offset_x, offset_y, slice_size):
    """
    Checks if a bounding box intersects with a slice and adjusts it if needed.
    """
    x, y, w, h = bbox
    x_max, y_max = x + w, y + h
    
    # Define the slice boundaries
    slice_x_min, slice_y_min = offset_x, offset_y
    slice_x_max, slice_y_max = offset_x + slice_size, offset_y + slice_size
    
    # Check intersection
    if not (x_max <= slice_x_min or x >= slice_x_max or y_max <= slice_y_min or y >= slice_y_max):
        # Calculate adjusted bounding box within slice boundaries
        new_x = max(x - offset_x, 0)
        new_y = max(y - offset_y, 0)
        new_w = min(w, slice_x_max - x) if x + w > slice_x_max else w
        new_h = min(h, slice_y_max - y) if y + h > slice_y_max else h
        print(f"Adjusted bbox within slice: [new_x={new_x}, new_y={new_y}, new_w={new_w}, new_h={new_h}]")
        return [new_x, new_y, new_w, new_h]
    
    print("No intersection with slice boundaries.")
    return None  # No intersection

In [ ]:
def calculate_slice_positions(image_width, image_height, slice_size, overlap_ratio):
    # Calculate overlap in pixels and effective stride
    overlap_pixels = int(slice_size * overlap_ratio)
    stride = slice_size - overlap_pixels

    # Generate slice positions for the current image
    slice_positions = []
    for y in range(0, image_height - slice_size + 1, stride):
        for x in range(0, image_width - slice_size + 1, stride):
            slice_positions.append((x, y))
    
    # Handle edge case: if the image dimensions are not exact multiples of stride
    if (image_width - slice_size) % stride != 0:
        for y in range(0, image_height - slice_size + 1, stride):
            slice_positions.append((image_width - slice_size, y))
    if (image_height - slice_size) % stride != 0:
        for x in range(0, image_width - slice_size + 1, stride):
            slice_positions.append((x, image_height - slice_size))
    if (image_width - slice_size) % stride != 0 and (image_height - slice_size) % stride != 0:
        slice_positions.append((image_width - slice_size, image_height - slice_size))
    
    return slice_positions

# Parameters for slicing
slice_size = 640
overlap_ratio = 0.2

# Example for one image
original_image_width, original_image_height = 4096, 2000  # Replace with each image's actual dimensions as needed
slice_positions = calculate_slice_positions(original_image_width, original_image_height, slice_size, overlap_ratio)

# Verify slice positions
print(f"Generated {len(slice_positions)} slice positions for image of size {original_image_width}x{original_image_height}")


Generated 32 slice positions for image of size 4096x2000


: 

In [94]:

# Loop through each slice and apply adjustments
for i, (slice_x, slice_y) in enumerate(slice_positions):  # slice_positions should be generated based on stride
    print(f"Processing slice {i} at position ({slice_x}, {slice_y}) with size {slice_size}")

    # Calculate slice boundaries
    slice_x_min = slice_x
    slice_y_min = slice_y
    slice_x_max = slice_x + slice_size
    slice_y_max = slice_y + slice_size

    # Loop through each bounding box
    for bbox in original_bboxes:
        bbox_x_min, bbox_y_min, bbox_width, bbox_height = bbox
        bbox_x_max = bbox_x_min + bbox_width
        bbox_y_max = bbox_y_min + bbox_height

        # Check for intersection with the current slice
        if (bbox_x_min < slice_x_max and bbox_x_max > slice_x_min and 
            bbox_y_min < slice_y_max and bbox_y_max > slice_y_min):
            
            # Calculate intersection coordinates
            adj_x_min = max(bbox_x_min, slice_x_min) - slice_x_min
            adj_y_min = max(bbox_y_min, slice_y_min) - slice_y_min
            adj_x_max = min(bbox_x_max, slice_x_max) - slice_x_min
            adj_y_max = min(bbox_y_max, slice_y_max) - slice_y_min

            adj_bbox = [adj_x_min, adj_y_min, adj_x_max - adj_x_min, adj_y_max - adj_y_min]

            print(f"  Adjusted bbox for slice {i}: {adj_bbox}")

            # Visualization logic for the adjusted bounding box
            plt.gca().add_patch(
                plt.Rectangle((adj_x_min, adj_y_min), adj_x_max - adj_x_min, adj_y_max - adj_y_min, 
                              linewidth=2, edgecolor='blue', facecolor='none')
            )

    # Show the current slice for visual verification
    plt.imshow(slice_image)  # Ensure `slice_image` corresponds to this slice
    plt.title(f"Slice {i} at ({slice_x}, {slice_y})")
    plt.show()

NameError: name 'slice_positions' is not defined

In [95]:

# Main processing loop for each sliced image
for img in sliced_annotations["images"]:
    img_id = img["id"]
    img_name = img["file_name"]

    # Extract offset values from filename (assuming the naming format includes them)
    name_parts = img_name.split('_')
    offset_x = int(name_parts[-5])
    offset_y = int(name_parts[-4])
    slice_size = int(name_parts[-1].split('.')[0])

    print(f"\nProcessing slice: {img_name} with offset ({offset_x}, {offset_y}) and size {slice_size}")

    # Add sliced image info to adjusted annotations
    adjusted_annotations["images"].append({
        "id": img_id,
        "file_name": img_name,
        "width": slice_size,
        "height": slice_size
    })

    # Process each annotation and check if it intersects with the current slice
    for ann in original_annotations["annotations"]:
        if ann["image_id"] == img_id:
            print(f"\nOriginal bbox: {ann['bbox']}")

            # Check intersection and adjust bbox if needed
            adjusted_bbox = check_and_adjust_bbox(ann["bbox"], offset_x, offset_y, slice_size)
            
            if adjusted_bbox:
                print(f"Adding adjusted bbox for slice {img_id}: {adjusted_bbox}")
                # Append adjusted annotation to the new annotations dictionary
                adjusted_annotations["annotations"].append({
                    "id": ann["id"],
                    "image_id": img_id,
                    "category_id": ann["category_id"],
                    "bbox": adjusted_bbox,
                    "area": adjusted_bbox[2] * adjusted_bbox[3],
                    "iscrowd": ann.get("iscrowd", 0)
                })


Processing slice: 01BE01_0_0_0_640_640.png with offset (0, 0) and size 640

Processing slice: 01BE01_0_512_0_1152_640.png with offset (0, 512) and size 640

Processing slice: 01BE01_0_1024_0_1664_640.png with offset (0, 1024) and size 640

Processing slice: 01BE01_0_1536_0_2176_640.png with offset (0, 1536) and size 640

Processing slice: 01BE01_0_2048_0_2688_640.png with offset (0, 2048) and size 640

Processing slice: 01BE01_0_2560_0_3200_640.png with offset (0, 2560) and size 640

Processing slice: 01BE01_0_3072_0_3712_640.png with offset (0, 3072) and size 640

Processing slice: 01BE01_0_3456_0_4096_640.png with offset (0, 3456) and size 640

Processing slice: 01BE01_0_0_512_640_1152.png with offset (0, 0) and size 1152

Processing slice: 01BE01_0_512_512_1152_1152.png with offset (0, 512) and size 1152

Processing slice: 01BE01_0_1024_512_1664_1152.png with offset (0, 1024) and size 1152

Processing slice: 01BE01_0_1536_512_2176_1152.png with offset (0, 1536) and size 1152

Proce

In [ ]:
# Assuming `annotations` is your list of bounding boxes
for annotation in annotations:
    original_bbox = annotation['bbox']
    x_min, y_min, width, height = original_bbox
    x_max, y_max = x_min + width, y_min + height

    # Check if the bounding box intersects with the current slice
    slice_x_min, slice_y_min = offset  # Offset of current slice
    slice_x_max, slice_y_max = slice_x_min + slice_size, slice_y_min + slice_size

    intersects = not (x_max < slice_x_min or x_min > slice_x_max or y_max < slice_y_min or y_min > slice_y_max)
    
    if intersects:
        print(f"Original bbox: {original_bbox}")
        print(f"Slice boundaries: x_min={slice_x_min}, y_min={slice_y_min}, x_max={slice_x_max}, y_max={slice_y_max}")
        
        # Adjust bounding box relative to the slice
        adj_x_min = max(x_min - slice_x_min, 0)
        adj_y_min = max(y_min - slice_y_min, 0)
        adj_x_max = min(x_max - slice_x_min, slice_size)
        adj_y_max = min(y_max - slice_y_min, slice_size)
        
        adjusted_bbox = [adj_x_min, adj_y_min, adj_x_max - adj_x_min, adj_y_max - adj_y_min]
        print(f"Adjusted bbox: {adjusted_bbox}")


In [84]:
# Save the adjusted annotations to a new JSON file
adjusted_annotations_path = "../data/adjusted_annotations.json"
with open(adjusted_annotations_path, "w") as f:
    json.dump(adjusted_annotations, f, indent=4)

print(f"Adjusted annotations saaved to {adjusted_annotations_path}")


Adjusted annotations saaved to ../data/adjusted_annotations.json


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

# Function to visualize a slice with its adjusted bounding boxes
def visualize_slice_with_boxes(slice_image_path, adjusted_bboxes, slice_size):
    # Load the image slice
    image = Image.open(slice_image_path).convert("RGB")
    draw = ImageDraw.Draw(image)

    # Draw each adjusted bounding box on the slice
    for bbox in adjusted_bboxes:
        x_min, y_min, width, height = bbox
        x_max, y_max = x_min + width, y_min + height
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)

    # Display the image with bounding boxes
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis("on")
    plt.show()

# Example usage with your bounding box data and a specific slice path
slice_path = "..\data\images_sliced.png"  # Replace with your actual slice image path
adjusted_bboxes = [
    # Replace this with your adjusted bounding box data for the slice
    [100, 120, 30, 40],  # Example bounding box
    # Add other bounding boxes here
]

# Visualize the slice with bounding boxes
visualize_slice_with_boxes(slice_path, adjusted_bboxes, slice_size=640)  # Adjust slice_size as needed


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\GitHub Repos\\mitl_detect_inspection\\preprocessing\\path_to_your_slice_image.png'